In [2]:
import pandas as pd
import numpy as np
pd.set_option("display.max_rows", 101)


In [3]:
#Welcome to my notebook.
#My analysis will look at which companies have received CARES Act/PPP stimulus checks in New York.
#It will then compare them with a dataset of layoffs by company.
#The aim is to see produce a story looking at how some companies got big checks despite laying people off, hopefully.

In [4]:
# First, I'll read in my data.
df_na = pd.read_csv('../data/warn_na.csv')

In [5]:
# I'm going to quickly rename df_na to master.
master = df_na

In [6]:
# Let's look under the hood
master.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1213 entries, 0 to 1212
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   notice_date      1213 non-null   object 
 1   event_number     1213 non-null   object 
 2   reason           1213 non-null   object 
 3   company          1213 non-null   object 
 4   address          1212 non-null   object 
 5   county           1213 non-null   object 
 6   phone            1213 non-null   object 
 7   business_type    1213 non-null   object 
 8   affected         1135 non-null   float64
 9   total_employees  480 non-null    float64
 10  layoff_date      1210 non-null   object 
 11  dislocation      1213 non-null   object 
 12  union            1213 non-null   object 
 13  classification   1213 non-null   object 
dtypes: float64(2), object(12)
memory usage: 132.8+ KB


In [7]:
#I'll begin by filtering this down to 'the goods', aka actually interesting stuff!

In [8]:
newdata = master[['notice_date','reason','company','affected','total_employees','business_type']]

In [9]:
# How does it look?
newdata

,notice_date,reason,company,affected,total_employees,business_type
0,4/27/2020,Temporary Plant Layoff,"OS Restaurant Services, LLC (Bloomin Brands- O...",174.0,174.0,Restaurant
1,4/27/2020,Temporary Plant Closing,"OS Restaurant Services, LLC (Bloomin Brands - ...",260.0,260.0,Restaurant
2,4/27/2020,Temporary Plant Closing,"OS Restaurant Services, LLC (Bloomin Brands - ...",239.0,239.0,Restaurant
3,4/27/2020,Temporary Plant Closing,"OS Restaurant Services, LLC (Bloomin Brands - ...",289.0,289.0,Restaurant
4,4/27/2020,Temporary Plant Closing,"OS Restaurant Services, LLC (Bloomin Brands - ...",154.0,154.0,Restaurant
...,...,...,...,...,...,...
1208,1/8/2020,Plant Closing,"Connected Ventures, LLC (CH Media)",39.0,39.0,Operates online content and retail properties
1209,1/6/2020,Plant Closing,Macy's Broadway Mall Store (Macy's Retail Hold...,155.0,155.0,Retail Store
1210,12/27/2019,Temporary Plant Closing,Wesley Gardens Nursing Home,132.0,132.0,Nursing Home
1211,12/30/2019,Plant Closing,"127 W. 43rd St. Chophouse, Inc. (Heartland Bre...",106.0,106.0,Restaurant


In [10]:
# It looks much better, now I need to do some type conversions.

newdata = newdata.fillna(0) #gets rid of na values
newdata['affected'] = newdata['affected'].astype(int)
newdata['total_employees'] = newdata['total_employees'].astype(int)  #lets me do math on the numerical values


In [11]:
# Ok lets see how many people got fired..

newdata['affected'].sum()



110485

In [12]:
# That's alot...what companies fired the most?
newdata.groupby(['company'])['affected'].agg('sum').sort_values(ascending=False).reset_index().head(30)

,company,affected
0,"Abercrombie & Fitch, abercrombie kids, Hollist...",2696
1,"OS Restaurant Services, LLC (Bloomin Brands - ...",2478
2,American Sales Management Organization LLC dba...,1321
3,Regal Cinemas,1004
4,Zara USA,918
5,"Mid Rockland Imaging Partners, Inc.",894
6,Icon Parking Holdings LLC (227 Locations in NYC),883
7,Acme Bus Corp.,814
8,"Yeled v'Yalda Early Childhood Center, Inc.",762
9,"DO & CO New York Catering, Inc.",744


In [13]:
# Renaming this layoff_data
layoff_data = newdata
newdata.groupby(['business_type'])['affected'].agg('sum').sort_values(ascending=False).reset_index().head(30)


,business_type,affected
0,Restaurant,40689
1,Hotel,15358
2,Catering,3217
3,Retail clothing store,2696
4,Retail,2057
5,Transportation,1634
6,Airline Sales,1321
7,Auto Dealership,1176
8,Cinemas,1004
9,Clothing Retail,918


In [14]:

# Ok, now I'm going to read in my other dataset of companies that have received big stimulus money from Cares/PPP. Data comes from here: https://covidstimuluswatch.org/

In [15]:
## Reading in the data
grants = pd.read_csv('../data/stimulus_watch_data.csv')

In [16]:
## Look under the hood!
grants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5779 entries, 0 to 5778
Data columns (total 32 columns):
 #   Column                                                                            Non-Null Count  Dtype  
---  ------                                                                            --------------  -----  
 0   Recipient Company                                                                 5779 non-null   object 
 1   Parent Company                                                                    5779 non-null   object 
 2   Award Date                                                                        5779 non-null   int64  
 3   Award Type                                                                        5779 non-null   object 
 4   Grant Amount                                                                      5779 non-null   object 
 5   Face Loan Amount                                                                  5779 non-null   object 
 6   

In [17]:
# Lot of columns there, many of which I don't need. Let's see how many companies are in NY.
grants['Parent Headquarters State'].value_counts().head()


New York      509
California    502
Florida       331
Texas         325
Illinois      259
Name: Parent Headquarters State, dtype: int64

In [18]:
# The most! Good to know. Ok, making a New York only dataframe. It should have 509 rows.

In [19]:
newyorkgrants = grants.loc[grants['Parent Headquarters State'] == "New York"].reset_index(drop=True)


In [20]:
# Now I am going to make a smaller dataframe by trimming unnecessary columns.
newyorkdata = newyorkgrants[['Grant Amount','Award Date','Parent Company','Parent Headquarters State','Ownership Structure','CEO Pay','Award Type']]

In [21]:
# Lets sort and take a look.
newyorkdata = newyorkdata.sort_values('Grant Amount', ascending=False).reset_index(drop=True)

In [22]:
## This dataset is interesting enough on its own ^^. Very cool. Excuse my talking to myself. Gonna check if I recognize any of them

## Looks like I goofed on the grant amount, needs to be type: integer
newyorkdata['Grant Amount'] = newyorkdata['Grant Amount'].str.replace('$','').str.replace(',','').astype(int)


In [23]:

## Need to resort by Grant Amount
grant_data = newyorkdata.sort_values('Grant Amount', ascending=False).reset_index(drop=True)
grant_data.to_csv('../output/newyorkgrant.csv' )

In [52]:
newyorkdata = newyorkdata.sort_values('Grant Amount', ascending=False).reset_index(drop=True)

In [53]:
newyorkdata

,Grant Amount,Award Date,Parent Company,Parent Headquarters State,Ownership Structure,CEO Pay,Award Type
0,685100000,20200423,JetBlue Airways,New York,publicly traded,"$3,955,523",grant
1,277653312,20200508,Northwell Health,New York,non-profit,$0,grant
2,203180447,20200508,New York University,New York,non-profit,$0,grant
3,156708592,20200508,Montefiore Medical Center,New York,non-profit,$0,grant
4,152747325,20200508,NewYork-Presbyterian,New York,non-profit,$0,grant
5,143251512,20200508,NewYork-Presbyterian,New York,non-profit,$0,grant
6,140754861,20200508,Mount Sinai Medical Center (NY),New York,non-profit,$0,grant
7,137531543,20200508,Northwell Health,New York,non-profit,$0,grant
8,131500615,20200508,Maimonides Medical Center,New York,non-profit,$0,grant
9,118647057,20200508,NewYork-Presbyterian,New York,non-profit,$0,grant
